In [4]:
import pandas as pd
import numpy as np

from pandas import *

from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import *

from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM
from keras.wrappers.scikit_learn import KerasClassifier
from keras import regularizers

from dummies_bins_test_train_cv import get_Xy_train_test, cross_validation_process
from clean_chess_game_log import main_cleanup
_,_,_ = main_cleanup('../data/dest.pgn')
df = pd.read_csv('../data/use_for_predictions.csv')

In [5]:

for num in np.random.random_integers(5, 30, (10,)):
    print(num)

16
16
20
25
14
19
8
26
18
22


/home/guess/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: This function is deprecated. Please call randint(5, 30 + 1) instead
  """Entry point for launching an IPython kernel.


In [2]:
X_train, X_test, y_train, y_test, X, y, df_clean = get_Xy_train_test(df, .97, .99)

y Shape: (2098,)


/home/guess/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/home/guess/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


X Shape: (2098, 30)
X_train Shape: (2048, 30)
X_test Shape: (50, 30)
y_train Shape: (2048,)
y_test Shape: (50,)


In [3]:
X_train = X_train.astype('float64')
X_test = X_test.astype('float64')
y_train = y_train.astype('int64')
y_test = y_test.astype('int64')

In [40]:
X_train.shape

(2046, 30)

In [41]:
X_test.shape

(52, 30)

In [42]:
y_train.shape

(2046,)

In [43]:
std_sclr = StandardScaler()
X_train = std_sclr.fit_transform(X_train)
X_test = std_sclr.fit_transform(X_test)

In [44]:
std_sclr.scale_

array([ 0.49963004, 10.14705119,  0.31948553,  0.31948553,  1.        ,
        1.        ,  1.        ,  0.23316069,  0.26646936,  0.36080121,
        0.19230769,  0.23316069,  0.23316069,  0.19230769,  0.19230769,
        0.31948553,  0.26646936,  0.31948553,  0.29480211,  0.13733516,
        0.19230769,  0.13733516,  1.        ,  1.        ,  0.37831376,
        0.3413123 ,  0.36080121,  0.40839924,  0.3413123 ,  0.26646936])

In [77]:
def _classifier():
    classifier = Sequential()

    classifier.add(Dense(units=64, kernel_initializer='uniform',
                         activation='relu', input_dim=30))

    classifier.add(
        Dense(units=256, kernel_initializer='uniform', activation='softmax'))

    classifier.add(
        Dense(units=128, kernel_initializer='uniform', activation='relu'))

    classifier.add(
        Dense(units=1, kernel_initializer='uniform', activation='sigmoid'))

    classifier.compile(
        optimizer='nadam', loss='binary_crossentropy', metrics=['accuracy'])

    return classifier

In [78]:
classifier = _classifier()

In [79]:
classifier.fit(X_train, y_train, batch_size=7, epochs=50,
               class_weight='balanced')

Epoch 1/50
2046/2046 [==============================] - 2s 1ms/step - loss: 0.6933 - acc: 0.4985
Epoch 2/50
2046/2046 [==============================] - 1s 680us/step - loss: 0.6544 - acc: 0.5948
Epoch 3/50
2046/2046 [==============================] - 1s 644us/step - loss: 0.5708 - acc: 0.7107
Epoch 4/50
2046/2046 [==============================] - 1s 658us/step - loss: 0.5542 - acc: 0.7146
Epoch 5/50
2046/2046 [==============================] - 1s 659us/step - loss: 0.5447 - acc: 0.7195
Epoch 6/50
2046/2046 [==============================] - 1s 660us/step - loss: 0.5396 - acc: 0.7239
Epoch 7/50
2046/2046 [==============================] - 1s 661us/step - loss: 0.5320 - acc: 0.7302
Epoch 8/50
2046/2046 [==============================] - 1s 721us/step - loss: 0.5265 - acc: 0.7307
Epoch 9/50
2046/2046 [==============================] - 1s 676us/step - loss: 0.5195 - acc: 0.7439
Epoch 10/50
2046/2046 [==============================] - 1s 672us/step - loss: 0.5154 - acc: 0.7400
Epoch 11/50

In [80]:
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)

In [81]:
cm = confusion_matrix(y_test, y_pred)
cm

array([[15,  8],
       [10, 19]])

In [82]:
print(f'{((cm[0][0]+cm[1][1])/cm.sum()*100).round(1)}%')

65.4%


In [85]:
def dropout_classifier(optimizer):
    classifier = Sequential()
    classifier.add(Dense(units=64, kernel_initializer='uniform',
                         activation='relu', input_dim=30,
                         kernel_regularizer=regularizers.l2(0.01),
                         activity_regularizer=regularizers.l1(0.01)))

    classifier.add(Dropout(rate=0.16))

    classifier.add(
        Dense(units=256, kernel_initializer='uniform', activation='softmax',
              kernel_regularizer=regularizers.l2(0.01),
              activity_regularizer=regularizers.l1(0.01)))

    classifier.add(Dropout(rate=0.08))

    classifier.add(
        Dense(units=128, kernel_initializer='uniform', activation='relu'))

    classifier.add(Dropout(rate=0.04))

    classifier.add(
        Dense(units=128, kernel_initializer='uniform', activation='softmax',
              kernel_regularizer=regularizers.l2(0.01),
              activity_regularizer=regularizers.l1(0.01)))

    classifier.add(
        Dense(units=1, kernel_initializer='uniform', activation='sigmoid'))

    classifier.compile(
        optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

    return classifier

In [97]:
bs_1 = randint(10,20)
bs_2 = randint(20,30)
ep_1 = randint(100,200)
ep_2 = randint(200,300)

parameters = {'batch_size': [bs_1, bs_2],
              'nb_epoch': [ep_1, ep_2],
              'optimizer': ['Nadam', 'Adamax']}

In [98]:
classifier = KerasClassifier(build_fn=dropout_classifier)

In [99]:
grid_search = GridSearchCV(classifier, parameters,
                           scoring='f1_micro', cv=5, n_jobs=-4)

In [100]:
print([bs_1, bs_2, ep_1, ep_2])
grid_search = grid_search.fit(X=X_train, y=y_train)

[10, 29, 144, 239]
Epoch 1/1
1636/1636 [==============================] - 4s 3ms/step - loss: 0.9624 - acc: 0.4939
Epoch 1/1
1637/1637 [==============================] - 4s 3ms/step - loss: 0.9606 - acc: 0.4985
Epoch 1/1
1637/1637 [==============================] - 5s 3ms/step - loss: 0.9623 - acc: 0.4875
Epoch 1/1
1637/1637 [==============================] - 5s 3ms/step - loss: 0.9636 - acc: 0.4777
Epoch 1/1


KeyboardInterrupt: 

In [23]:
best_params = grid_search.best_params_
best_params

{'batch_size': 11,
 'metrics': 'accuracy',
 'nb_epoch': 136,
 'optimizer': 'Nadam'}

In [24]:
best_accu = grid_search.best_score_
best_accu

0.5

In [25]:
grid_search.cv_results_

/home/guess/anaconda3/lib/python3.6/site-packages/sklearn/utils/deprecation.py:125: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/home/guess/anaconda3/lib/python3.6/site-packages/sklearn/utils/deprecation.py:125: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/home/guess/anaconda3/lib/python3.6/site-packages/sklearn/utils/deprecation.py:125: FutureWarning: You are accessing a training score ('split2_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/home/guess/anaconda3/lib/python3.

{'mean_fit_time': array([1.95626278, 1.86280422, 2.04237003, 2.65089693, 2.50077305,
        2.76881981, 3.51045456, 3.13306255, 3.35499706, 3.9417161 ,
        3.77045817, 4.10874844, 4.59338212, 4.4006001 , 4.83682075,
        5.30044928, 5.13736792, 5.32125216, 5.8378161 , 5.51505432,
        5.84917669, 6.35840449, 6.23469224, 6.40556731, 6.94099956,
        6.91315556, 7.0387198 , 7.59981046, 7.75728326, 7.8697196 ,
        8.61835566, 7.92901921, 8.53506799, 8.75249362, 8.59890614,
        9.75399041]),
 'std_fit_time': array([0.0407468 , 0.09310205, 0.08139113, 0.10323613, 0.23038569,
        0.17639263, 0.24815985, 0.17819387, 0.1059951 , 0.24145006,
        0.22014575, 0.25223405, 0.11204237, 0.04961857, 0.29187926,
        0.23819855, 0.19555931, 0.19253297, 0.33804709, 0.38337115,
        0.19191702, 0.23797548, 0.53996041, 0.11535254, 0.06349024,
        0.18512041, 0.06298417, 0.13247473, 0.55428582, 0.16007322,
        0.20718979, 0.05266484, 0.21150767, 0.09837604, 0.062

In [26]:
y_pred = grid_search.predict(X_test)
y_pred = (y_pred > 0.5)

In [27]:
cm = confusion_matrix(y_test, y_pred)
cm

array([[106,   0],
       [108,   0]])

In [28]:
print(f'classification_report:\n{classification_report(y_test, y_pred)}')
print('---------------------------------------------------------------\n\n')
print(f'average_precision_score:\n{average_precision_score(y_test, y_pred)*100}%')
print('---------------------------------------------------------------\n\n')
print(f'f1_score:\n{f1_score(y_test, y_pred)*100}%')
print('---------------------------------------------------------------\n\n')
print(f'accuracy_score:\n{accuracy_score(y_test, y_pred)*100}%')
print('---------------------------------------------------------------\n\n')
print(f'cohen_kappa_score:\n{cohen_kappa_score(y_test, y_pred)}')
print('---------------------------------------------------------------\n\n')
print(f'roc_curve:\n{roc_curve(y_test, y_pred)}')
print('---------------------------------------------------------------\n\n')
print(f'roc_auc_score:\n{roc_auc_score(y_test, y_pred)*100}%')
print('---------------------------------------------------------------\n\n')
print(f'brier_score_loss:\n{brier_score_loss(y_test, y_pred)*100}%')
print('---------------------------------------------------------------\n\n')
print(f'recall_score:\n{recall_score(y_test, y_pred)*100}%')

classification_report:
              precision    recall  f1-score   support

           0       0.50      1.00      0.66       106
           1       0.00      0.00      0.00       108

   micro avg       0.50      0.50      0.50       214
   macro avg       0.25      0.50      0.33       214
weighted avg       0.25      0.50      0.33       214

---------------------------------------------------------------


average_precision_score:
50.467289719626166%
---------------------------------------------------------------


f1_score:
0.0%
---------------------------------------------------------------


accuracy_score:
49.532710280373834%
---------------------------------------------------------------


cohen_kappa_score:
0.0
---------------------------------------------------------------


roc_curve:
(array([0., 1.]), array([0., 1.]), array([1, 0]))
---------------------------------------------------------------


roc_auc_score:
50.0%
-----------------------------------------------------

/home/guess/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/guess/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


In [29]:
print(f'Accuracy = {round((cm[0][0]+cm[1][1])/(cm.sum())/.01,1)}%')

Accuracy = 72.4%


In [ ]:
def dropout_classifier_set():
    classifier = Sequential()
    classifier.add(Dense(units=5, kernel_initializer='uniform',
                         activation='relu', input_dim=5))
    classifier.add(Dropout(rate=0.12))
    classifier.add(
        Dense(units=40, kernel_initializer='uniform', activation='relu'))
    classifier.add(Dropout(rate=0.08))
    classifier.add(
        Dense(units=8, kernel_initializer='uniform', activation='relu'))
    classifier.add(Dropout(rate=0.04))
    classifier.add(
        Dense(units=1, kernel_initializer='uniform', activation='sigmoid'))

    classifier.compile(
        optimizer='Nadam', loss='binary_crossentropy', metrics=['accuracy'])
    return classifier

In [ ]:
# classifier = dropout_classifier()

In [ ]:
classifier.fit(X_train, y_train, batch_size=13, 
                   epochs=250, shuffle=False)

In [ ]:
batch_list = [134, 67, 28, 14, 7, 4]
for num in batch_list:
    classifier.fit(X_train, y_train, batch_size=num, 
                   epochs=100, shuffle=False)

In [ ]:
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)

In [ ]:
cm = confusion_matrix(y_test, y_pred)
cm

In [ ]:
print(f'Accuracy = {(cm[0][0]+cm[1][1])/(cm.sum())/.01}%')

### 75.5% Accuracy on 1824 training, 200 test
__3 layers__
- layer 1 = ReLU X 7
- layer 2 = SoftMax X 14
- layer 3 = ReLU X 7
- output layer = Sigmoid X 1

__metrics__ = Cosine

__loss__ = binary_crossentropy

__Optimizer__ = Nadam

__Batch size__ = 12

__Epochs__ = 175

no __diff__

In [ ]:
classifier = KerasClassifier(build_fn=_classifier, batch_size=12, epochs=200)

In [ ]:
accus = cross_val_score(estimator=classifier, X=X_train,
                        y=y_train, cv=10, n_jobs=-2, scoring='accuracy')
accus

In [ ]:
accu_mean = accus.mean()
accu_mean

In [ ]:
accu_var = accus.std()
accu_var

In [ ]:
accus

In [ ]:
def dropout_classifier(optimizer, metrics):
    classifier = Sequential()
    classifier.add(Dense(units=5, kernel_initializer='uniform',
                         activation='relu', input_dim=5))
    classifier.add(Dropout(rate=0.12))
    classifier.add(
        Dense(units=40, kernel_initializer='uniform', activation='relu'))
    classifier.add(Dropout(rate=0.08))
    classifier.add(
        Dense(units=8, kernel_initializer='uniform', activation='relu'))
    classifier.add(Dropout(rate=0.04))
    classifier.add(
        Dense(units=1, kernel_initializer='uniform', activation='sigmoid'))

    classifier.compile(
        optimizer=optimizer, loss='binary_crossentropy', metrics=[metrics])
    return classifier